In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools
from scipy.optimize import linear_sum_assignment
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical # used for converting labels to one-hot-encoding
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input,Dense, Dropout, Flatten, Conv2D, MaxPool2D,Conv2DTranspose,Reshape
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import itertools
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [2]:
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
y_test = [np.argmax(y)for y in y_test]
y_train = [np.argmax(y)for y in y_train]
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

In [3]:
def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_sum_assignment(w.max() - w)
    ind = np.asarray(ind)
    ind = np.transpose(ind)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [4]:
n_clusters = len(np.unique(y))
def DeepCluster(input_shape=(75, 100, 3), filters=[32, 64, 128, 7]):
    input_img = Input(shape=input_shape)

    x = Conv2D(filters[0],kernel_size=(3, 3), padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)
    x = Conv2D(filters[0], kernel_size=(3, 3), padding='same', activation='relu', name='conv2')(x)
    x = MaxPool2D(pool_size = (2, 2))(x)
    
    x = Conv2D(filters[1], kernel_size=(3, 3), padding='same', activation='relu', name='conv3')(x)
    x = Conv2D(filters[1], kernel_size=(3, 3), padding='same', activation='relu', name='conv4')(x)
    x = MaxPool2D(pool_size = (2, 2))(x)
    
    x = Conv2D(filters[2], kernel_size=(3, 3), padding='same', activation='relu', name='conv5')(x)
    x = Conv2D(filters[2], kernel_size=(3, 3), padding='same', activation='relu', name='conv6')(x)
    x = MaxPool2D(pool_size = (2, 2))(x)
    
    x = Flatten()(x)
    conv_output = Dense(filters[2], activation='relu')(x)
    classifier = Dense(filters[3], activation='softmax')(conv_output)
    return  [Model(inputs=input_img, outputs=conv_output, name='conv_output'),Model(inputs=input_img, outputs=classifier, name='classifier')]

In [5]:
conv_net, Deep_cluster = DeepCluster()
conv_net.summary()
Deep_cluster.summary()

Model: "conv_output"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 75, 100, 3)]      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 75, 100, 32)       896       
_________________________________________________________________
conv2 (Conv2D)               (None, 75, 100, 32)       9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 37, 50, 32)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 37, 50, 64)        18496     
_________________________________________________________________
conv4 (Conv2D)               (None, 37, 50, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 25, 64)        

In [13]:
Deep_cluster.compile(optimizer=tf.keras.optimizers.SGD(0.01, 0.9), loss='kld')

In [14]:
def target_distribution(q):
    temp =1/(1+q**2)
    temp=temp.transpose()/temp.sum(1)
    temp=temp.transpose()
    return temp

In [17]:
from sklearn.decomposition import PCA
x_1 = x.reshape((x.shape[0], -1))
pca = PCA(n_components=128)
x_2 = pca.fit_transform(x_1)
kmeans = KMeans(n_clusters=n_clusters)
y_pred_last = kmeans.fit_predict(x_2)
acc(y,y_pred_last)

0.3175237144283575

In [18]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
nmi = normalized_mutual_info_score
ari = adjusted_rand_score
print(n_clusters)
loss = 0
index = 0
maxiter = 3000
batch_size = 10
update_interval = 50
index_array = np.arange(x.shape[0])

for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        temp_feature = conv_net.predict(x, verbose=0)
        y_pred = kmeans.predict(temp_feature)
        q = kmeans.transform(temp_feature)
        p = target_distribution(q)# update the auxiliary target distribution p
        # evaluate the clustering performance
        if y is not None:
            loss = np.round(loss, 5)
            print('Iter %d:  loss=%f' % (ite,loss))
        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if (ite>0 and loss<0.00002):
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = Deep_cluster.train_on_batch(x=x[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

Deep_cluster.save_weights('./Deep_cluster_model_final.h5')

7
Iter 0:  loss=0.000000
Iter 50:  loss=0.000140
Iter 100:  loss=0.000050
Iter 150:  loss=0.000030
Iter 200:  loss=0.000030
Iter 250:  loss=0.000030
Iter 300:  loss=0.000030
Iter 350:  loss=0.000030
Iter 400:  loss=0.000010


In [19]:
# Eval.
q = Deep_cluster.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p
# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    acc_value = np.round(acc(y, y_pred), 5)
    nmi_value = np.round(nmi(y, y_pred), 5)
    ari_value = np.round(ari(y, y_pred), 5)
    loss_value = np.round(loss, 5)
    print('Acc = %.5f, nmi = %.5f, ari = %.5f' % (acc_value, nmi_value, ari_value), ' ; loss=', loss)

Acc = 0.66950, nmi = 5.87500, ari = 0.00000  ; loss= 1e-05


/home/wangligt/environments/tf2_py3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
